In [7]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import zipfile
from lxml import etree

In [9]:
pptx_path="/Users/ecsde/Desktop/industrial_group_project/data/raw/Zoekwoordenonderzoek .docx"
output_folder="/Users/ecsde/Desktop/industrial_group_project/data/clean"
with zipfile.ZipFile(pptx_path, 'r') as pptx:
    pptx.extractall(output_folder)


In [10]:
xml_content

NameError: name 'xml_content' is not defined

In [9]:
tree

<Element {http://schemas.openxmlformats.org/presentationml/2006/main}sld at 0x1113b3640>

In [18]:
ns={
    'a': 'http://schemas.openxmlformats.org/drawingml/2006/main',
    'r': 'http://schemas.openxmlformats.org/officeDocument/2006/relationships',
    'p': 'http://schemas.openxmlformats.org/presentationml/2006/main'
}
for information in tree.xpath('//p:sp', namespaces=ns):
    text=information.xpath('.//a:t', namespaces=ns)
    ph_type=information.xpath('.//p:ph/@type', namespaces=ns)
    font=information.xpath('.//a:rPr/@typeface', namespaces=ns)
    size=information.xpath('.//a:rPr/@sz', namespaces=ns)

    print(text)

    if ph_type:
        print(f"Placeholder type: {ph_type[0]} | Text: {''.join(etree.tostring(n, method='text', encoding='unicode') for n in text)} | Font: {font[0] if font else 'N/A'} | Size: {int(size[0])/100 if size else 'N/A'} pt")
    else:
        print(f"Non-placeholder | Text: {''.join(etree.tostring(n, method='text', encoding='unicode') for n in text)} | Font: {font[0] if font else 'N/A'} | Size: {int(size[0])/100 if size else 'N/A'} pt")

[<Element {http://schemas.openxmlformats.org/drawingml/2006/main}t at 0x1114b1680>]
Placeholder type: ctrTitle | Text: Greenhouse Management Project | Font: N/A | Size: N/A pt
[<Element {http://schemas.openxmlformats.org/drawingml/2006/main}t at 0x11154f780>]
Placeholder type: subTitle | Text: Kha Pham Van | Font: N/A | Size: 14.0 pt


### Problem identify

The above problem of can not identify the font or the size of the text is because for the first text, it's font and size doesn't being defined inline (inside of the slideN.xml)</br></br>Tracing it via viewing the slideN.xml.rels to see which file is it linking with, from that I found out the font and size of the first text as well as the font of the second text are in the slideLayouts folder

In [21]:
namespace={'a': 'http://schemas.openxmlformats.org/drawingml/2006/main'}
text_text=tree.xpath('.//a:t/text()', namespaces=namespace)
print(text_text)
print(etree.tostring(n, method='text', encoding='unicode') for n in text_text)


for i in tree.xpath('//a:r', namespaces=namespace):
    text2=i.xpath('.//a:t', namespaces=namespace)
    size2=i.xpath('.//a:rPr/@sz', namespaces=namespace)
    print(text2)
    print(size2)

['Greenhouse Management Project', 'Kha Pham Van']
<generator object <genexpr> at 0x1116558a0>
[<Element {http://schemas.openxmlformats.org/drawingml/2006/main}t at 0x1114b1680>]
[]
[<Element {http://schemas.openxmlformats.org/drawingml/2006/main}t at 0x11154f780>]
['1400']


### Discovery of the above code

When pulling features out from the loop, meaning I have to access the outer layer first. So than for extracting text feature, if do it in a loop the output will be a found at a destination, but not the text itself. Therefore, what I have to do is to transform it back to text and join those text back into a string, since converting back to text will return a list not a string. </br> </br> Extracting directly from the exact tag will result in text directly

## Try to extract features from the Layout layer or Master layer

Starting all again from the starting point</br>And also looping through every slides, so I can have every slides information

In [23]:
pptx_path="/Users/nguyenanhvu/Documents/AI_Basic_Fontys/Portfolio/GroupProject/FirstMeetingLamecoPresentation.pptx"
output_folder="/Users/nguyenanhvu/Documents/AI_Basic_Fontys/ExtractPowerPointFeatures/Output2"
with zipfile.ZipFile(pptx_path, 'r') as pptx:
    xml_content= pptx.read('ppt/slides/slide1.xml')
    tree = etree.fromstring(xml_content)
    pptx.extractall(output_folder)

In [21]:
from lxml import etree
import os

ns = {
    'a': 'http://schemas.openxmlformats.org/drawingml/2006/main',
    'r': 'http://schemas.openxmlformats.org/officeDocument/2006/relationships',
    'p': 'http://schemas.openxmlformats.org/presentationml/2006/main'
}

def extract_text_styles(tree, slide_path, ppt_root):
    for shape in tree.xpath('//p:sp', namespaces=ns):
        text_nodes = shape.xpath('.//a:t', namespaces=ns)
        text_content = ''.join(etree.tostring(n, method='text', encoding='unicode') for n in text_nodes)
        if not text_content.strip():
            continue

        # Inline text properties
        font = shape.xpath('.//a:rPr//a:latin/@typeface', namespaces=ns)
        size = shape.xpath('.//a:rPr/@sz', namespaces=ns)
        bold = shape.xpath('.//a:rPr/@b', namespaces=ns)
        italic = shape.xpath('.//a:rPr/@i', namespaces=ns)
        underline = shape.xpath('.//a:rPr/@u', namespaces=ns)

        # Placeholder type (if available)
        ph_type = shape.xpath('.//p:ph/@type', namespaces=ns)

        # --- FALLBACK: if font/size missing, trace to layout/master ---
        if not font or not size:
            layout_font, layout_size = trace_to_layout_master(slide_path, ppt_root, ph_type)
            font = font or [layout_font]
            size = size or [layout_size]

        # --- Print results ---
        print(
            f"Text: {text_content} | "
            f"Font: {font[0] if font else 'N/A'} | "
            f"Size: {int(size[0])/100 if size and size[0] else 'N/A'} pt | "
            f"Bold: {bool(bold)} | Italic: {bool(italic)} | Underline: {bool(underline)} | "
            f"Placeholder: {ph_type[0] if ph_type else 'Non-placeholder'}"
        )


def trace_to_layout_master(slide_path, ppt_root, ph_type):
    """Trace to layout → master for missing font/size"""
    # Step 1: find layout
    layout_path = get_linked_file(slide_path, "slideLayout", ppt_root)
    if layout_path and os.path.exists(layout_path):
        layout_tree = etree.parse(layout_path)
        rPr = layout_tree.xpath('.//a:defRPr', namespaces=ns)
        if rPr:
            font, size = extract_from_rPr(rPr[0])
            if font or size:
                return font, size

    # Step 2: find master
    if layout_path:
        master_path = get_linked_file(layout_path, "slideMaster", ppt_root)
        if master_path and os.path.exists(master_path):
            master_tree = etree.parse(master_path)
            rPr = master_tree.xpath('.//a:defRPr', namespaces=ns)
            if rPr:
                font, size = extract_from_rPr(rPr[0])
                return font, size

    return None, None


def extract_from_rPr(rPr):
    """Extract font + size attributes from <a:rPr> or <a:defRPr>"""
    font = None
    size = None
    latin = rPr.xpath('.//a:latin/@typeface', namespaces=ns)
    buFont = rPr.xpath('.//a:buFont/@typeface', namespaces=ns)
    if latin:
        font = latin[0]
    elif buFont:
        font = buFont[0]
    if 'sz' in rPr.attrib:
        size = rPr.get('sz')
    return font, size


def get_linked_file(xml_path, link_type, ppt_root):
    """Find the layout/master file linked from .rels"""
    rels_path = xml_path.replace(".xml", ".xml.rels")
    if not os.path.exists(rels_path):
        return None

    rel_tree = etree.parse(rels_path)
    target = rel_tree.xpath(
        f'//rel:Relationship[contains(@Type, "{link_type}")]/@Target',
        namespaces={'rel': 'http://schemas.openxmlformats.org/package/2006/relationships'}
    )
    if not target:
        return None

    target_path = os.path.join(os.path.dirname(xml_path), target[0])
    target_path = os.path.normpath(target_path.replace("..", ppt_root))
    return target_path

In [25]:
import os
from lxml import etree

ppt_root = "/Users/nguyenanhvu/Documents/AI_Basic_Fontys/ExtractPowerPointFeatures/Output/ppt"
slides_dir = os.path.join(ppt_root, "slides")

for file in sorted(os.listdir(slides_dir)):
    if file.startswith("slide") and file.endswith(".xml"):
        slide_path= os.path.join(slides_dir, file)
        print(f"\n -- Analyzing {file} -- \n")
        tree = etree.parse(slide_path)
        extract_text_styles(tree, slide_path,ppt_root)



 -- Analyzing slide1.xml -- 

Text: Greenhouse Management Project
                         | Font: None | Size: N/A pt | Bold: False | Italic: False | Underline: False | Placeholder: ctrTitle
Text: Kha Pham Van
                         | Font: None | Size: 14.0 pt | Bold: False | Italic: False | Underline: False | Placeholder: subTitle

 -- Analyzing slide2.xml -- 

Text: Terraced Agriculture | Font: None | Size: N/A pt | Bold: False | Italic: False | Underline: False | Placeholder: title

 -- Analyzing slide3.xml -- 


 -- Analyzing slide4.xml -- 


 -- Analyzing slide5.xml -- 

Text: 100 employees | Font: None | Size: N/A pt | Bold: False | Italic: False | Underline: False | Placeholder: Non-placeholder
Text: 13 employees | Font: None | Size: N/A pt | Bold: False | Italic: False | Underline: False | Placeholder: Non-placeholder

 -- Analyzing slide6.xml -- 

Text: Automatic shading and warming Greenhouse | Font: None | Size: N/A pt | Bold: False | Italic: False | Underline: False | 